In [227]:
from fuzzywuzzy import fuzz

In [37]:
import pandas as pd
import numpy as np

<h2> Remedy for NaN industry code </h2>

In [266]:
df = pd.read_csv("metadata_reports_noduplicates_with_industry.csv")

In [143]:
df_industry_mapping = pd.read_csv("Ticker_ConpanyName_Industry.csv")
df_industry_mapping = df_industry_mapping.dropna(subset=['ggroup', "conm", "TICKER"]).reset_index(drop=True)

In [134]:
companies_ref = df_industry_mapping.conm.str.upper().str.strip().unique()

In [135]:
ticker_ref = df_industry_mapping.TICKER.str.upper().str.strip().unique()

In [136]:
no_industry_index = df['industry'].index[df['industry'].apply(np.isnan)].tolist()

In [137]:
df_no_industry = df.iloc[df['industry'].index[df['industry'].apply(np.isnan)],:]

In [228]:
def set_row(df, index, row_ref):
    df.iloc[i, -3] = int(row_ref["ggroup"])
    df.iloc[i, -4] = int(row_ref["gsector"])
    df.iloc[i, -5] = int(row_ref["PERMNO"])

In [243]:
fixed_index = []
not_fixed_index = []
for i in no_industry_index:
    row = df.iloc[i, :]
    company_name = str(row["Company Name"]).upper().strip()
    ticker = str(row["TICKER"]).strip()
    # hard coded roles
    if ticker == "BRKa":
        ticker = "BRK"
    if ticker == "HPE (1)":
        ticker = "HPE"
    if ticker == "BFb":
        ticker = "BF"
    if company_name in companies_ref:
        latest_year = df_industry_mapping[df_industry_mapping.conm == company_name]["year"].max()
        row_ref = df_industry_mapping[(df_industry_mapping.conm == company_name) & (df_industry_mapping.year == latest_year)].max()
        set_row(df, i, row_ref)
        fixed_index.append(i)
    elif ticker in ticker_ref:
        latest_year = df_industry_mapping[df_industry_mapping.TICKER == ticker]["year"].max()
        row_ref = df_industry_mapping[(df_industry_mapping.TICKER == ticker) & (df_industry_mapping.year == latest_year)].max()
        set_row(df, i, row_ref)
        fixed_index.append(i)
    else:
        ### approximate matching ###
        found = False
        for ref_company in companies_ref:
            if ref_company is np.nan:
                continue
            if fuzz.ratio(ref_company, company_name) > 90:
                print(ref_company+ " V.S " +company_name)
                latest_year = df_industry_mapping[df_industry_mapping.conm == ref_company]["year"].max()
                row_ref = df_industry_mapping[(df_industry_mapping.conm == ref_company) & (df_industry_mapping.year == latest_year)].max()
                set_row(df, i, row_ref)
                fixed_index.append(i)
                found = True
                break
        if not found:
            not_fixed_index.append(i)

WENDY'S CO V.S WENDYS CO
GRID DYNAMICS HOLDGS INC V.S GRID DYNAMICS HOLDINGS INC
GRID DYNAMICS HOLDGS INC V.S GRID DYNAMICS HOLDINGS INC
AUTOMATIC DATA PROCESSING V.S AUTOMATIC DATA PROCESSING INC
AUTOMATIC DATA PROCESSING V.S AUTOMATIC DATA PROCESSING INC
ATN INTERNATIONAL INC V.S MAGNA INTERNATIONAL INC
U S SILICA HOLDINGS INC V.S U.S. SILICA HOLDINGS INC
NATIONAL GENERAL HOLDINGS CP V.S NATIONAL GENERAL HOLDINGS CORP
ARENA PHARMACEUTICALS INC V.S AURINIA PHARMACEUTICALS INC
HERON THERAPEUTICS INC V.S HARPOON THERAPEUTICS INC
CALLON PETROLEUM CO/DE V.S CALLON PETROLEUM CO
C H ROBINSON WORLDWIDE INC V.S CH ROBINSON WORLDWIDE INC
CONSTELLATION BRANDS V.S CONSTELLATION BRANDS INC
NEXTIER OILFIELD SOLUTNS INC V.S NEXTIER OILFIELD SOLUTIONS INC
HELIX ENERGY SOLUTIONS GROUP V.S HELIX ENERGY SOLUTIONS GROUP INC
OCEANEERING INTERNATIONAL V.S OCEANEERING INTERNATIONAL INC
PARK HOTELS & RESORTS V.S PARK HOTELS & RESORTS INC
PARK HOTELS & RESORTS V.S PARK HOTELS & RESORTS INC
AFFILIATED MANAGER

In [244]:
len(fixed_index)

2770

In [245]:
len(not_fixed_index)

9082

In [246]:
df.iloc[not_fixed_index].TICKER.unique()

array(['ACN', 'AIV', 'ALLE', 'AMCR', 'AMT', 'AON', 'APTV', 'ARE', 'AVB',
       'BXP', 'CB', 'CCI', 'CCL', 'DLR', 'DRE', 'EI', 'EQIX', 'EQR',
       'ESS', 'ETN', 'EXR', 'FRT', 'FTI', 'GRMN', 'HST', 'INFO', 'IRM',
       'IVZ', 'JCI', 'KIM', 'LIN', 'LYB', 'MAA', 'MDT', 'MYL', 'NCLH',
       'NLSN', 'O', 'PEAK', 'PLD', 'PNR', 'PRGO', 'PSA', 'RCL', 'RE',
       'REG', 'SBAC', 'SLB', 'SLG', 'SPG', 'STE', 'STX', 'TEL', 'TT',
       'UDR', 'VNO', 'VTR', 'WELL', 'WLTW', 'WY'], dtype=object)

In [247]:
Russell_ticker_list = pd.read_excel("./list_SP500.xlsx", sheet_name="Russell 3000")[".RUA"].str.split('.').str[0].tolist()
SP500_ticker_list = pd.read_excel("./list_SP500.xlsx", sheet_name="S&P500")[".SP500"].str.split('.').str[0].tolist()

In [248]:
not_in_russell = []
in_russell = []
not_in_SP500 = []
in_SP500 = []
for ticker in df.iloc[not_fixed_index].TICKER.unique():
    if ticker not in Russell_ticker_list:
        not_in_russell.append(ticker)
    else:
        in_russell.append(ticker)
    if ticker not in SP500_ticker_list:
        not_in_SP500.append(ticker)
    else:
        in_SP500.append(ticker)

In [249]:
in_SP500

['ACN',
 'AIV',
 'ALLE',
 'AMCR',
 'AMT',
 'AON',
 'APTV',
 'ARE',
 'AVB',
 'BXP',
 'CB',
 'CCI',
 'CCL',
 'DLR',
 'DRE',
 'EQIX',
 'EQR',
 'ESS',
 'ETN',
 'EXR',
 'FRT',
 'FTI',
 'GRMN',
 'HST',
 'INFO',
 'IRM',
 'IVZ',
 'JCI',
 'KIM',
 'LIN',
 'LYB',
 'MAA',
 'MDT',
 'MYL',
 'NCLH',
 'NLSN',
 'O',
 'PEAK',
 'PLD',
 'PNR',
 'PRGO',
 'PSA',
 'RCL',
 'RE',
 'REG',
 'SBAC',
 'SLB',
 'SLG',
 'SPG',
 'STE',
 'STX',
 'TEL',
 'TT',
 'UDR',
 'VNO',
 'VTR',
 'WELL',
 'WLTW',
 'WY']

In [250]:
not_in_russell

['AMCR', 'EI', 'FTI', 'STX', 'TEL']

In [251]:
not_in_SP500

['EI']

In [252]:
df.iloc[not_fixed_index].groupby(["TICKER"]).count().reset_index().sort_values(by=['Available'], ascending=False)

,TICKER,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,...,SecondaryTicker,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup,industry,quarter_year
22,FTI,363,363,363,363,352,111,340,363,361,...,363,363,349,363,363,0,0,0,0,363
0,ACN,294,294,294,294,292,85,288,294,294,...,294,294,272,294,294,0,0,0,0,294
15,EI,290,290,290,290,287,31,283,290,290,...,290,290,277,290,290,0,0,0,0,290
33,MDT,290,290,290,290,287,40,282,290,290,...,290,290,286,290,290,0,0,0,0,290
47,SLB,249,249,249,249,249,16,246,249,249,...,249,249,230,249,249,0,0,0,0,249
6,APTV,227,227,227,227,211,53,216,227,227,...,227,227,224,227,227,1,1,1,0,227
35,NCLH,222,222,222,222,215,103,185,222,222,...,222,222,221,222,222,0,0,0,0,222
34,MYL,218,218,218,218,214,49,209,218,218,...,218,218,195,218,218,0,0,0,0,218
51,STX,216,216,216,216,216,27,204,216,216,...,216,216,200,216,216,0,0,0,0,216
39,PLD,202,202,202,202,202,32,188,202,202,...,202,202,191,202,202,0,0,0,0,202


<h2> Dropping rows with ticker company mismatch <h2>

In [269]:
df = pd.read_csv("metadata_reports_noduplicates_full_industry.csv")

In [270]:
df['industry'] = df['ggroup']
df['quarter_year'] = df["year"].astype("str") + " q" + df["quarter"].astype("str")
df2= df.dropna(subset=['industry']).reset_index(drop=True)
df2['industry-quarter'] = list(zip(df2.industry, df2.quarter_year))
list_industries_quarter = df2.groupby('industry-quarter').count().reset_index()['industry-quarter'].tolist()

In [272]:
temp=df2.groupby(['TICKER','Company Name']).count()['Available'].reset_index()
temp2=temp.groupby(['TICKER']).apply(lambda x: x.sort_values(['Available'], ascending=False))
del temp2['TICKER']
temp2=temp2.reset_index()
temp3=temp2.groupby('TICKER').first().reset_index()

In [273]:
index_no_issue_list = []
for i in range(df2.shape[0]):
    row = df2.iloc[i, :]
    ticker = str(row["TICKER"]).split(".")[0]
    company_name = str(row["Company Name"])
    if company_name == max(temp3[temp3["TICKER"] == ticker]["Company Name"]):
        index_no_issue_list.append(i)

In [274]:
df_no_issue = df2.iloc[index_no_issue_list]

In [275]:
df_no_issue

,Unnamed: 0,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,...,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup,industry,quarter_year,industry-quarter
0,0,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Increase, TargetPrice Increase","3Q18 Recap: Maintaining Our Rating, Raising Ou...",9.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
1,1,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,Morningstar | Raising Our FVE After American S...,13.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
2,2,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, TargetPrice Increase",Morningstar | Rising Fuel Costs Raise American...,25.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
3,3,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,3Q18 Recap; Strategic Initiatives Aimed At Dri...,5.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
4,4,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, Recommendation Decrease, Ta...",American Airlines Group Inc.,9.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98729,98729,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation - 4Q Core EPS $0.80 - So...,12.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"
98730,98730,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Solid Fees Offset Slightly Smaller Balance She...,7.0,...,1,NaN,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"
98731,98731,2018-02-05,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,4Q17 - Post Conference Call Model Update,7.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"
98732,98732,2018-01-17,2018-01-10,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation,9.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"


In [277]:
df.to_csv("cleaned_metadata_reports_noduplicates_with_industry.csv")

In [84]:
# fixed_list = []
# for i in index_issue_list:
#     row = df2.iloc[i, :]
#     if not row["Prim. Ticker"] is np.nan:
#         primary_tickers = row["Prim. Ticker"].split(",")
#     else:
#         continue
#     found = False
#     for p_ticker in primary_tickers:
#         if p_ticker.split(".")[0] == str(row["TICKER"]):
#             found = True
#             break
#     if found:
#         fixed_list.append(i)